In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image

# For Troponin Datset

## Load latest Troponin data (8April)

- information include all troponin reading during ICU stay, weight ,data categorization for disease, first creatinine reading
- Need to add in admittime

In [2]:
# Load dataset

df_2 = pd.read_csv('Troponin_inICU_8April.csv')

# df_2 
# 1723 rows × 205 columns

# Convert ICU intime to keep only the first ICU stays for each patients
# Convert ICU intime 
df_2['intime_1'] = pd.to_datetime(df_2['intime_1'],format='%d/%m/%Y %H:%M')

# sort by ICU intime
df_2.sort_values(by='intime_1',ascending=True,inplace=True)
# df_2
# 1723 rows × 205 columns

In [3]:
# Keep only the first hospitalization and ICU stay for each patients to obtain distinct subject_id

df_2 = df_2.groupby('subject_id').first().reset_index()
df_2

,subject_id,hadm_id,stay_id,first_careunit_1,last_careunit_1,intime_1,outtime_1,charttime_1,storetime_1,flag_1,...,Atrial_fibrillation,hypertension,Ischemic_heart_disease,diabetes,Cerebrovascular,Admission Weight (Kg),creatinine_storetime_1,creatinine_value_1,creatinine_charttime_1,value_max
0,10002155,20345487,32358465,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-03-09 21:33:00,10/3/2131 18:09,10/3/2131 2:04,10/3/2131 3:17,abnormal,...,0,0,0,0,0,21.1,9/3/2131 23:46,1.4,9/3/2131 23:02,0.05
1,10020944,29974575,30757476,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-02-27 16:40:00,8/3/2131 18:30,27/2/2131 18:00,27/2/2131 20:36,abnormal,...,0,0,0,0,0,107.0,27/2/2131 19:03,1.4,27/2/2131 18:00,0.05
2,10026255,22059910,31248398,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2201-07-07 19:40:00,8/7/2201 15:43,10/7/2201 8:30,10/7/2201 12:21,None,...,1,1,0,0,0,68.6,8/7/2201 5:55,0.5,8/7/2201 4:46,0.02
3,10029057,28919637,36004746,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2187-07-30 21:48:00,3/8/2187 3:17,31/7/2187 5:18,31/7/2187 6:22,abnormal,...,1,1,0,0,0,69.6,31/7/2187 6:22,0.9,31/7/2187 5:18,0.06
4,10034317,20827960,33536886,Trauma SICU (TSICU),Trauma SICU (TSICU),2159-08-30 12:35:00,31/8/2159 16:26,26/8/2159 19:10,26/8/2159 19:58,None,...,0,0,0,0,0,NaN,26/8/2159 14:56,1.1,26/8/2159 13:10,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,19985545,21516111,32154634,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2144-07-22 02:06:00,23/7/2144 15:05,7/8/2144 2:49,8/8/2144 4:12,abnormal,...,0,0,0,0,0,66.7,21/7/2144 22:21,1.5,21/7/2144 21:31,0.04
1627,19986604,21846283,36243531,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2159-11-03 18:48:00,4/11/2159 19:56,3/11/2159 23:04,4/11/2159 1:04,abnormal,...,0,0,0,0,0,65.4,4/11/2159 6:52,0.6,4/11/2159 5:51,0.04
1628,19987702,26568899,34859288,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2131-01-15 22:29:00,17/1/2131 21:26,13/1/2131 21:25,13/1/2131 22:10,abnormal,...,0,1,0,0,0,76.9,13/1/2131 22:10,1.1,13/1/2131 21:25,0.33
1629,19996783,21880161,31700331,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2188-05-10 02:20:00,10/5/2188 22:30,9/5/2188 21:56,9/5/2188 23:47,abnormal,...,0,1,0,0,0,58.5,9/5/2188 23:35,1.1,9/5/2188 21:56,3.84


##### Add admittime

In [4]:
# Add admittime 
'''
Below codes are copied directly from the previous section admission
'''

# Load admission time data
admission = pd.read_csv('Hospital admission time.csv')
# 10849 rows

# Only keep admission time for relevant stay_id
admission = admission.loc[admission['stay_id'].isin(df_2['stay_id'])]
# Only 1631 rows left

# Keep only subject_id,hadm_id,stay_id and admission time 
admission = admission[['subject_id','hadm_id','stay_id', 'admittime']]
admission.shape

(1631, 4)

In [5]:
df_2 = pd.merge(df_2,admission,how='left',on=['subject_id','hadm_id','stay_id'])
df_2.shape

(1631, 206)

##### Add First troponin charttime. ** First troponin test here is the first troponin measured throughout the entire hospitalization

In [7]:
# Load data to obtain the first troponin after hospital admission 
FirstTroponin = pd.read_csv('Troponin_All hospitalization_6April.csv',usecols=['subject_id','hadm_id','stay_id','charttime_First'])
FirstTroponin

,subject_id,hadm_id,stay_id,charttime_First
0,10002155,20345487,32358465,10/3/2131 2:04
1,10020944,29974575,30757476,27/2/2131 18:00
2,10026255,22059910,31248398,10/7/2201 8:30
3,10029057,28919637,36004746,31/7/2187 5:18
4,10034317,20827960,33536886,26/8/2159 19:10
...,...,...,...,...
1718,19985545,21516111,32154634,7/8/2144 2:49
1719,19986604,21846283,36243531,3/11/2159 23:04
1720,19987702,26568899,34859288,13/1/2131 21:25
1721,19996783,21880161,31700331,9/5/2188 21:56


In [8]:
# Add this information to df_2
df_2 = pd.merge(df_2,FirstTroponin,how='left',on=['subject_id','hadm_id','stay_id'])
df_2

,subject_id,hadm_id,stay_id,first_careunit_1,last_careunit_1,intime_1,outtime_1,charttime_1,storetime_1,flag_1,...,Ischemic_heart_disease,diabetes,Cerebrovascular,Admission Weight (Kg),creatinine_storetime_1,creatinine_value_1,creatinine_charttime_1,value_max,admittime,charttime_First
0,10002155,20345487,32358465,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-03-09 21:33:00,10/3/2131 18:09,10/3/2131 2:04,10/3/2131 3:17,abnormal,...,0,0,0,21.1,9/3/2131 23:46,1.4,9/3/2131 23:02,0.05,2131-03-09 20:33:00,10/3/2131 2:04
1,10020944,29974575,30757476,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-02-27 16:40:00,8/3/2131 18:30,27/2/2131 18:00,27/2/2131 20:36,abnormal,...,0,0,0,107.0,27/2/2131 19:03,1.4,27/2/2131 18:00,0.05,2131-02-27 15:34:00,27/2/2131 18:00
2,10026255,22059910,31248398,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2201-07-07 19:40:00,8/7/2201 15:43,10/7/2201 8:30,10/7/2201 12:21,None,...,0,0,0,68.6,8/7/2201 5:55,0.5,8/7/2201 4:46,0.02,2201-07-07 18:15:00,10/7/2201 8:30
3,10029057,28919637,36004746,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2187-07-30 21:48:00,3/8/2187 3:17,31/7/2187 5:18,31/7/2187 6:22,abnormal,...,0,0,0,69.6,31/7/2187 6:22,0.9,31/7/2187 5:18,0.06,2187-07-30 20:43:00,31/7/2187 5:18
4,10034317,20827960,33536886,Trauma SICU (TSICU),Trauma SICU (TSICU),2159-08-30 12:35:00,31/8/2159 16:26,26/8/2159 19:10,26/8/2159 19:58,None,...,0,0,0,NaN,26/8/2159 14:56,1.1,26/8/2159 13:10,0.02,2159-08-26 00:00:00,26/8/2159 19:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,19985545,21516111,32154634,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2144-07-22 02:06:00,23/7/2144 15:05,7/8/2144 2:49,8/8/2144 4:12,abnormal,...,0,0,0,66.7,21/7/2144 22:21,1.5,21/7/2144 21:31,0.04,2144-07-22 00:25:00,7/8/2144 2:49
1627,19986604,21846283,36243531,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2159-11-03 18:48:00,4/11/2159 19:56,3/11/2159 23:04,4/11/2159 1:04,abnormal,...,0,0,0,65.4,4/11/2159 6:52,0.6,4/11/2159 5:51,0.04,2159-11-03 17:16:00,3/11/2159 23:04
1628,19987702,26568899,34859288,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2131-01-15 22:29:00,17/1/2131 21:26,13/1/2131 21:25,13/1/2131 22:10,abnormal,...,0,0,0,76.9,13/1/2131 22:10,1.1,13/1/2131 21:25,0.33,2131-01-13 16:45:00,13/1/2131 21:25
1629,19996783,21880161,31700331,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2188-05-10 02:20:00,10/5/2188 22:30,9/5/2188 21:56,9/5/2188 23:47,abnormal,...,0,0,0,58.5,9/5/2188 23:35,1.1,9/5/2188 21:56,3.84,2188-05-09 15:55:00,9/5/2188 21:56


### Add statin prescription information

In [10]:
# Load statin presription information
df_statin = pd.read_csv('ICU_statin rx.csv')
df_statin

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,subject_id-2,hadm_id-2,...,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,16307213,23083430,35175902,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2172/11/18 22:20,2172/11/21 14:13,2.661505,16307213.0,23083430.0,...,51785.0,6.068703e+10,20mg Tablet,NaN,20.0,mg,1.0,TAB,1.0,PO
1,13303843,21519524,37677563,Trauma SICU (TSICU),Trauma SICU (TSICU),2149/10/27 14:54,2149/11/3 16:56,7.085220,13303843.0,21519524.0,...,16579.0,5.107905e+10,40mg Tablet,NaN,80.0,mg,2.0,TAB,1.0,PO/NG
2,13303843,21519524,37677563,Trauma SICU (TSICU),Trauma SICU (TSICU),2149/10/27 14:54,2149/11/3 16:56,7.085220,13303843.0,21519524.0,...,16579.0,5.107905e+10,40mg Tablet,NaN,80.0,mg,2.0,TAB,1.0,PO/NG
3,13303843,21519524,37677563,Trauma SICU (TSICU),Trauma SICU (TSICU),2149/10/27 14:54,2149/11/3 16:56,7.085220,13303843.0,21519524.0,...,16577.0,5.107905e+10,10mg Tablet,NaN,10.0,mg,1.0,TAB,1.0,PO/NG
4,15568805,26593415,39771468,Neuro Stepdown,Neuro Stepdown,2135/9/15 3:48,2135/9/18 11:44,3.330938,15568805.0,26593415.0,...,29967.0,6.808401e+10,10mg Tablet,NaN,10.0,mg,1.0,TAB,1.0,PO/NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,15846970,22132828,38637564,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2168/12/2 8:24,2168/12/3 22:03,1.569225,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11975,13552776,23747995,32423949,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2117/7/24 20:50,2117/7/26 8:48,1.498553,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11976,11782473,27260782,32141710,Trauma SICU (TSICU),Trauma SICU (TSICU),2157/11/26 11:52,2157/11/27 18:37,1.281262,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11977,16441343,29859744,39177305,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2167/12/12 14:51,2167/12/14 17:01,2.090023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Convert intime 
df_statin['intime'] = pd.to_datetime(df_statin['intime'],format='%Y/%m/%d %H:%M')
df_statin['starttime'] = pd.to_datetime(df_statin['starttime'],format='%Y/%m/%d %H:%M')

In [12]:
# We will only keep subject_id which appears in our df_2 (i.e. keep statin prescription information for all relevant patients)
# Filter by patients
df_statin = df_statin[df_statin.subject_id.isin(df_2['subject_id'])]
df_statin.sort_values(by=['subject_id','intime','starttime'], inplace=True, ascending=True)
df_statin.shape
# 2986 rows

(2986, 25)

In [13]:
# We will also add the hospital admittime which will be used later
# Load admission time data again
admission = pd.read_csv('Hospital admission time.csv')
df_statin = pd.merge(df_statin,admission[['subject_id','hadm_id','stay_id','admittime']],how='left',on=['subject_id','hadm_id','stay_id'])
df_statin.shape

(2986, 26)

In [14]:
# Convert admittimt format
df_statin['admittime'] = pd.to_datetime(df_statin['admittime'],format='%Y/%m/%d %H:%M')
df_statin.shape

(2986, 26)

In [16]:
# Match statin_rx with bigger dataset based on 'stay_id' to understand statin rx during the ICU stay
statin_rx = df_statin.loc[df_statin['stay_id'].isin(df_2['stay_id'])]
statin_rx.shape # (1925, 26)

(1925, 26)

In [17]:
# Split original dataset into those with statins prescribed vs. without 

no_statins = statin_rx.loc[statin_rx['subject_id-2'].isnull()]
# no_statins.shape
# 996 rows 

statins = statin_rx.dropna(subset=['subject_id-2'])
# statins.shape
# 929 rows 

In [18]:
print(no_statins.shape)
print(statins.shape)

(996, 26)
(929, 26)


In [19]:
# Check whether there are overlaps between the no_statins and statins group based on subject_id
overlap = pd.merge(no_statins,statins,how='inner',on=['subject_id'])
overlap.shape
# We confirm that there is no overlap between no_statins patient groups and statin patients

(0, 51)

##### Patients with statin

In [20]:
# For each stay_id(patient), we will only keep the first statin prescription information
statins = statins.sort_values(by='starttime',ascending=True).groupby('stay_id').first().reset_index()
statins.shape
# (635, 26)

(635, 26)

In [21]:
# Keep only the relevant columns 
statins = statins[['subject_id', 'hadm_id', 'stay_id','intime','starttime','admittime']]

# For statins group we will add one more column 'statin' and label it as 2
# 2 only means that patients received statin during the current hospitalization (hadm_id)
statins['statin'] = 2
statins


,subject_id,hadm_id,stay_id,intime,starttime,admittime,statin
0,15841119,21924009,30019415,2134-06-14 21:28:00,2134-06-15 20:00:00,2134-06-14 21:27:00,2
1,13257593,21537254,30030289,2151-11-07 13:22:00,2151-11-07 20:00:00,2151-11-07 13:21:00,2
2,17542553,20715579,30031362,2173-03-17 09:04:00,2173-03-17 20:00:00,2173-03-17 08:25:00,2
3,18667723,20531077,30036908,2155-03-29 00:05:00,2155-03-29 22:00:00,2155-03-28 23:22:00,2
4,14428378,21698209,30046705,2181-01-27 22:00:00,2181-01-05 10:00:00,2181-01-05 15:15:00,2
...,...,...,...,...,...,...,...
630,19747837,22979536,39890790,2181-09-19 00:57:00,2181-09-19 10:00:00,2181-09-18 23:50:00,2
631,19481923,26414869,39894026,2176-03-23 01:42:00,2176-04-16 20:00:00,2176-03-23 00:57:00,2
632,17567629,29763926,39914721,2167-12-24 01:14:00,2167-12-24 20:00:00,2167-12-23 22:35:00,2
633,14648756,27461599,39915070,2175-05-21 18:00:00,2175-05-21 10:00:00,2175-05-21 16:26:00,2


##### Patients without Statin

For these patients, we will need to check that whether there is statin precribed prior to or during the current ICU stay (e.g. from previous hospitalization).

Label patients as Group 0 if:
- Patients did not receive any statin during the current hospitalization or all previous hospitalization


In [22]:
# Check whether there is duplicates in no_statins
no_statins.to_csv('no statins.csv',index=False)

'''
After checking, there is no duplicates in no_statins dataset, thus we can obtain a list of patients (subject_id and stay_id) with no statin records
'''
NoStatins_inICU = no_statins[['subject_id', 'hadm_id', 'stay_id','intime','admittime']]
NoStatins_inICU.rename(columns={'intime':'ICUintime','admittime':'ICUadmittime'},inplace=True)
NoStatins_inICU.shape # Contains the DISTINCT list of patient information who have not been prescribed with statin during current ICU stay

(996, 5)

In [23]:
'''
Filter patients who have statin prescribed in other hospitalizations.

Create a temp dataset containing all the Rx prescription situation for those patients (based on subject_id) - who received statin in other hospitalizaiotns but not the current one.
From original df_statin dataset (2986 rows)

By doing so, we obtain all statin prescription history of patients who had statin prescribed in other hospitalizations but not during the current hospitalization.
'''

temp = df_statin.loc[df_statin['subject_id'].isin(NoStatins_inICU['subject_id'])]
temp = temp.dropna(subset=['subject_id-2']) # Drop cases where no statin is prescribed as we are only interested to know whether statin prescribed
temp.sort_values(by=['subject_id','intime'],ascending=True,inplace=True)
# Add ICUintime ICUadmittime (for current ICU stay) from NoStatins_inICU for comparison later
temp = pd.merge(temp,NoStatins_inICU[['subject_id','ICUintime','ICUadmittime']],how='left',on='subject_id')
temp.to_csv('temp.csv',index=False)
temp.shape
(57, 28)

(57, 28)

In [24]:
'''
Filter patients who have statin prescribed in PREVIOUS hospitalizations.

On the temp dataset, we will use groupby by subject_id
For all the rows related to a particular subject_id, we will only keep rows where the admittime is earlier than the ICUadmittime in our Troponin dataset.
This means that  this statin Rx happens in previous hospitalization (vs. current ICU stay) -> patients acutally have prior exposure to statin
'''

temp_group = temp.groupby('subject_id')

cols = temp.columns

# temp_df is created to store the data 
temp_df = pd.DataFrame(columns=cols)


# In the loop -> we will only keep a row if intime is earlier than the ICUintime
for name, group in temp_group:
    group = group.loc[group['admittime']<group['ICUadmittime']] # Must be previous hospitalization 
    temp_df = pd.concat([temp_df,group],axis=0)
temp_df.shape # (31, 28)


(31, 28)

In [25]:
count = pd.DataFrame(temp_df['subject_id'].value_counts())
count.shape # 17 Distinct patients

(17, 1)

In [27]:
# For patient in temp_df, they should be classified as 1 (statin-exposed group) since they have already received statin prior to current hospitalization
# The rest will be non-exposed group as they 

no_statins = no_statins[['subject_id', 'hadm_id', 'stay_id','intime','starttime','admittime']]

no_statins['statin'] = np.where(no_statins['subject_id'].isin(temp_df['subject_id']),1,0)
no_statins.shape # (996, 7)

(996, 7)

In [28]:
no_statins['statin'].value_counts()

0    979
1     17
Name: statin, dtype: int64

##### Merge all data together to have one final statin_rx dataset

Note that 2 here only means that patients received statins during current hospitalization, but we have not classify patients based on our classification criteria (e.g. compare statin time vs. hospitalization time)

In [29]:
rx = pd.concat([statins,no_statins],axis=0).drop(columns=['admittime'])
rx # 1631 DISTINCT rows

,subject_id,hadm_id,stay_id,intime,starttime,statin
0,15841119,21924009,30019415,2134-06-14 21:28:00,2134-06-15 20:00:00,2
1,13257593,21537254,30030289,2151-11-07 13:22:00,2151-11-07 20:00:00,2
2,17542553,20715579,30031362,2173-03-17 09:04:00,2173-03-17 20:00:00,2
3,18667723,20531077,30036908,2155-03-29 00:05:00,2155-03-29 22:00:00,2
4,14428378,21698209,30046705,2181-01-27 22:00:00,2181-01-05 10:00:00,2
...,...,...,...,...,...,...
2968,19960115,28341607,30600396,2113-12-03 03:51:00,NaT,0
2975,19972172,23625602,39366389,2156-10-18 20:27:00,NaT,0
2979,19985545,21516111,32154634,2144-07-22 02:06:00,NaT,0
2981,19986604,21846283,36243531,2159-11-03 18:48:00,NaT,0


In [30]:
rx['statin'].value_counts()

0    979
2    635
1     17
Name: statin, dtype: int64

### Add statin prescription information to df_2 dataset

In [31]:
df_2

,subject_id,hadm_id,stay_id,first_careunit_1,last_careunit_1,intime_1,outtime_1,charttime_1,storetime_1,flag_1,...,Ischemic_heart_disease,diabetes,Cerebrovascular,Admission Weight (Kg),creatinine_storetime_1,creatinine_value_1,creatinine_charttime_1,value_max,admittime,charttime_First
0,10002155,20345487,32358465,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-03-09 21:33:00,10/3/2131 18:09,10/3/2131 2:04,10/3/2131 3:17,abnormal,...,0,0,0,21.1,9/3/2131 23:46,1.4,9/3/2131 23:02,0.05,2131-03-09 20:33:00,10/3/2131 2:04
1,10020944,29974575,30757476,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-02-27 16:40:00,8/3/2131 18:30,27/2/2131 18:00,27/2/2131 20:36,abnormal,...,0,0,0,107.0,27/2/2131 19:03,1.4,27/2/2131 18:00,0.05,2131-02-27 15:34:00,27/2/2131 18:00
2,10026255,22059910,31248398,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2201-07-07 19:40:00,8/7/2201 15:43,10/7/2201 8:30,10/7/2201 12:21,None,...,0,0,0,68.6,8/7/2201 5:55,0.5,8/7/2201 4:46,0.02,2201-07-07 18:15:00,10/7/2201 8:30
3,10029057,28919637,36004746,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2187-07-30 21:48:00,3/8/2187 3:17,31/7/2187 5:18,31/7/2187 6:22,abnormal,...,0,0,0,69.6,31/7/2187 6:22,0.9,31/7/2187 5:18,0.06,2187-07-30 20:43:00,31/7/2187 5:18
4,10034317,20827960,33536886,Trauma SICU (TSICU),Trauma SICU (TSICU),2159-08-30 12:35:00,31/8/2159 16:26,26/8/2159 19:10,26/8/2159 19:58,None,...,0,0,0,NaN,26/8/2159 14:56,1.1,26/8/2159 13:10,0.02,2159-08-26 00:00:00,26/8/2159 19:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,19985545,21516111,32154634,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2144-07-22 02:06:00,23/7/2144 15:05,7/8/2144 2:49,8/8/2144 4:12,abnormal,...,0,0,0,66.7,21/7/2144 22:21,1.5,21/7/2144 21:31,0.04,2144-07-22 00:25:00,7/8/2144 2:49
1627,19986604,21846283,36243531,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2159-11-03 18:48:00,4/11/2159 19:56,3/11/2159 23:04,4/11/2159 1:04,abnormal,...,0,0,0,65.4,4/11/2159 6:52,0.6,4/11/2159 5:51,0.04,2159-11-03 17:16:00,3/11/2159 23:04
1628,19987702,26568899,34859288,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2131-01-15 22:29:00,17/1/2131 21:26,13/1/2131 21:25,13/1/2131 22:10,abnormal,...,0,0,0,76.9,13/1/2131 22:10,1.1,13/1/2131 21:25,0.33,2131-01-13 16:45:00,13/1/2131 21:25
1629,19996783,21880161,31700331,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2188-05-10 02:20:00,10/5/2188 22:30,9/5/2188 21:56,9/5/2188 23:47,abnormal,...,0,0,0,58.5,9/5/2188 23:35,1.1,9/5/2188 21:56,3.84,2188-05-09 15:55:00,9/5/2188 21:56


In [32]:
rx

,subject_id,hadm_id,stay_id,intime,starttime,statin
0,15841119,21924009,30019415,2134-06-14 21:28:00,2134-06-15 20:00:00,2
1,13257593,21537254,30030289,2151-11-07 13:22:00,2151-11-07 20:00:00,2
2,17542553,20715579,30031362,2173-03-17 09:04:00,2173-03-17 20:00:00,2
3,18667723,20531077,30036908,2155-03-29 00:05:00,2155-03-29 22:00:00,2
4,14428378,21698209,30046705,2181-01-27 22:00:00,2181-01-05 10:00:00,2
...,...,...,...,...,...,...
2968,19960115,28341607,30600396,2113-12-03 03:51:00,NaT,0
2975,19972172,23625602,39366389,2156-10-18 20:27:00,NaT,0
2979,19985545,21516111,32154634,2144-07-22 02:06:00,NaT,0
2981,19986604,21846283,36243531,2159-11-03 18:48:00,NaT,0


In [33]:
# Add statin prescription information 

df_3 = pd.merge(df_2,rx,how='left',on=['subject_id','hadm_id','stay_id'])

In [34]:
df_3['statin'].value_counts()

0    979
2    635
1     17
Name: statin, dtype: int64

In [36]:
df_3

,subject_id,hadm_id,stay_id,first_careunit_1,last_careunit_1,intime_1,outtime_1,charttime_1,storetime_1,flag_1,...,Admission Weight (Kg),creatinine_storetime_1,creatinine_value_1,creatinine_charttime_1,value_max,admittime,charttime_First,intime,starttime,statin
0,10002155,20345487,32358465,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-03-09 21:33:00,10/3/2131 18:09,10/3/2131 2:04,10/3/2131 3:17,abnormal,...,21.1,9/3/2131 23:46,1.4,9/3/2131 23:02,0.05,2131-03-09 20:33:00,10/3/2131 2:04,2131-03-09 21:33:00,2131-03-09 10:00:00,2
1,10020944,29974575,30757476,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-02-27 16:40:00,8/3/2131 18:30,27/2/2131 18:00,27/2/2131 20:36,abnormal,...,107.0,27/2/2131 19:03,1.4,27/2/2131 18:00,0.05,2131-02-27 15:34:00,27/2/2131 18:00,2131-02-27 16:40:00,NaT,0
2,10026255,22059910,31248398,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2201-07-07 19:40:00,8/7/2201 15:43,10/7/2201 8:30,10/7/2201 12:21,None,...,68.6,8/7/2201 5:55,0.5,8/7/2201 4:46,0.02,2201-07-07 18:15:00,10/7/2201 8:30,2201-07-07 19:40:00,2201-07-07 21:00:00,2
3,10029057,28919637,36004746,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2187-07-30 21:48:00,3/8/2187 3:17,31/7/2187 5:18,31/7/2187 6:22,abnormal,...,69.6,31/7/2187 6:22,0.9,31/7/2187 5:18,0.06,2187-07-30 20:43:00,31/7/2187 5:18,2187-07-30 21:48:00,2187-07-30 10:00:00,2
4,10034317,20827960,33536886,Trauma SICU (TSICU),Trauma SICU (TSICU),2159-08-30 12:35:00,31/8/2159 16:26,26/8/2159 19:10,26/8/2159 19:58,None,...,NaN,26/8/2159 14:56,1.1,26/8/2159 13:10,0.02,2159-08-26 00:00:00,26/8/2159 19:10,2159-08-30 12:35:00,2159-08-30 20:00:00,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,19985545,21516111,32154634,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2144-07-22 02:06:00,23/7/2144 15:05,7/8/2144 2:49,8/8/2144 4:12,abnormal,...,66.7,21/7/2144 22:21,1.5,21/7/2144 21:31,0.04,2144-07-22 00:25:00,7/8/2144 2:49,2144-07-22 02:06:00,NaT,0
1627,19986604,21846283,36243531,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2159-11-03 18:48:00,4/11/2159 19:56,3/11/2159 23:04,4/11/2159 1:04,abnormal,...,65.4,4/11/2159 6:52,0.6,4/11/2159 5:51,0.04,2159-11-03 17:16:00,3/11/2159 23:04,2159-11-03 18:48:00,NaT,0
1628,19987702,26568899,34859288,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2131-01-15 22:29:00,17/1/2131 21:26,13/1/2131 21:25,13/1/2131 22:10,abnormal,...,76.9,13/1/2131 22:10,1.1,13/1/2131 21:25,0.33,2131-01-13 16:45:00,13/1/2131 21:25,2131-01-15 22:29:00,2131-01-13 20:00:00,2
1629,19996783,21880161,31700331,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2188-05-10 02:20:00,10/5/2188 22:30,9/5/2188 21:56,9/5/2188 23:47,abnormal,...,58.5,9/5/2188 23:35,1.1,9/5/2188 21:56,3.84,2188-05-09 15:55:00,9/5/2188 21:56,2188-05-10 02:20:00,2188-05-10 00:00:00,2


#### Further classify Group 2 patients 

##### Label some Group 2 patients as Group 1 patients (statin exposed)

To be qualified as Group 1 (statin exposed group), we perform the following:
- Take all patients who were prescribed statin within 24hrs of admission
- From these patients, select for patients who had first troponin done at least 48 hours after admission. In this case, we can safely say that the statin prescription had likely nothing to do with the troponin test being done. 

Steps:
1. Filter for 'statin' labeled as 2
2. Filter for rows with  'starttime'-'admittime' <=24 hours
3. Filter for rows with 'charttime_First'-'admittime' >=48 hours. NOTE that charttime_First refers to the chart time of the first Troponin after hospitalization, but after ICU!
4. The final subset of patients will be labelled as 1 

In [37]:
# Convert relevant time  to datetime format and for all the others

df_3['charttime_First'] = pd.to_datetime(df_3['charttime_First'],format='%d/%m/%Y %H:%M')
df_3['starttime'] = pd.to_datetime(df_3['starttime'],format='%Y/%m/%d %H:%M:%S')
df_3['admittime'] = pd.to_datetime(df_3['admittime'],format='%Y/%m/%d %H:%M:%S')

In [38]:
# 1. Filter for 'statin' labeled as 2
subset = df_3.loc[df_3['statin']==2]
subset.shape #(635, 210)

(635, 210)

In [39]:
subset[['charttime_First','starttime','admittime']].head(10)

,charttime_First,starttime,admittime
0,2131-03-10 02:04:00,2131-03-09 10:00:00,2131-03-09 20:33:00
2,2201-07-10 08:30:00,2201-07-07 21:00:00,2201-07-07 18:15:00
3,2187-07-31 05:18:00,2187-07-30 10:00:00,2187-07-30 20:43:00
4,2159-08-26 19:10:00,2159-08-30 20:00:00,2159-08-26 00:00:00
9,2158-01-05 05:24:00,2158-01-04 20:00:00,2158-01-04 21:01:00
11,2193-08-21 05:14:00,2193-08-20 21:00:00,2193-08-20 18:12:00
13,2130-01-27 22:35:00,2130-01-27 10:00:00,2130-01-27 17:26:00
16,2129-04-18 05:45:00,2129-04-10 10:00:00,2129-04-10 15:41:00
20,2167-06-14 21:20:00,2167-06-14 10:00:00,2167-06-14 17:53:00
22,2168-09-22 07:15:00,2168-09-22 11:00:00,2168-09-17 07:15:00


In [40]:
# 2. Filter for rows with  'starttime'-'admittime' <=24 hours
subset['rx difference_duration'] = subset['starttime'] - subset['admittime']
subset['rx difference_duration'] = subset['rx difference_duration']/np.timedelta64(1,'h')

subset = subset.loc[subset['rx difference_duration']<=24]
subset.shape # (482, 211)

(482, 211)

In [41]:
# 3. Filter for rows with 'charttime_First'-'admittime' >=48 hours
subset['troponin diff'] = subset['charttime_First'] - subset['admittime']
subset['troponin diff'] = subset['troponin diff']/np.timedelta64(1,'h')
subset = subset.loc[subset['troponin diff']>=48]
subset.shape # (104, 212)

(104, 212)

In [43]:
# Update the statin label from 2 to 1 for these group of patients 
df_3['statin'] = np.where(df_3['subject_id'].isin(subset['subject_id']),1,df_3['statin'])

In [44]:
df_3['statin'].value_counts()

0    979
2    531
1    121
Name: statin, dtype: int64

##### Reclassify the REMAINING group 2 patients to Group 1 by looking at their statin exposure in previous hospitalization

For remaining Group 2:
1. Look at statin prescription history from previous hospitalization -> filter patients who have statin prescribed during previous hospitalizaiton 

In [48]:
# Create a temp subset for the remaining Group 2 patients

temp_subset = df_3.loc[df_3['statin']==2]
# temp_subset
# 531 rows × 210 columns

In [49]:
# Rename some columns for easier comparison
temp_subset.rename(columns={'intime':'ICUintime','admittime':'ICUadmittime'},inplace=True)

In [50]:
df_statin 
# 2986 rows × 26 columns -> only relevant subject_id

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,subject_id-2,hadm_id-2,...,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,admittime
0,10002155,20345487,32358465,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-03-09 21:33:00,2131/3/10 18:09,0.858576,10002155.0,20345487.0,...,7.101574e+07,40mg Tablet,NaN,80.0,mg,2.0,TAB,1.0,PO/NG,2131-03-09 20:33:00
1,10020944,29974575,30757476,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-02-27 16:40:00,2131/3/8 18:30,9.076829,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2131-02-27 15:34:00
2,10026255,22059910,31248398,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2201-07-07 19:40:00,2201/7/8 15:43,0.835590,10026255.0,22059910.0,...,6.808406e+10,10mg Tablet,NaN,10.0,mg,1.0,TAB,1.0,PO/NG,2201-07-07 18:15:00
3,10026255,22059910,38229329,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2201-07-10 10:10:00,2201/7/11 13:48,1.150868,10026255.0,22059910.0,...,6.808406e+10,10mg Tablet,NaN,10.0,mg,1.0,TAB,1.0,PO/NG,2201-07-07 18:15:00
4,10029057,28919637,36004746,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2187-07-30 21:48:00,2187/8/3 3:17,3.228947,10029057.0,28919637.0,...,7.101564e+07,20mg Tablet,NaN,20.0,mg,1.0,TAB,1.0,PO/NG,2187-07-30 20:43:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2981,19986604,21846283,36243531,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2159-11-03 18:48:00,2159/11/4 19:56,1.047836,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-11-03 17:16:00
2982,19987702,26568899,34859288,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2131-01-15 22:29:00,2131/1/17 21:26,1.956030,19987702.0,26568899.0,...,5.107902e+10,80mg Tablet,NaN,80.0,mg,1.0,TAB,1.0,PO/NG,2131-01-13 16:45:00
2983,19996783,21880161,31700331,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2188-05-10 02:20:00,2188/5/10 22:30,0.840116,19996783.0,21880161.0,...,5.107902e+10,80mg Tablet,NaN,80.0,mg,1.0,TAB,1.0,PO/NG,2188-05-09 15:55:00
2984,19996783,21880161,35175329,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2188-05-11 11:59:00,2188/5/13 21:13,2.384907,19996783.0,21880161.0,...,5.107902e+10,80mg Tablet,NaN,80.0,mg,1.0,TAB,1.0,PO/NG,2188-05-09 15:55:00


In [51]:
'''
Filter remaining Group 2 patients who have statin prescribed in other hospitalizations.
'''

temp_2 = df_statin.loc[df_statin['subject_id'].isin(temp_subset['subject_id'])] #(1075, 26)
temp_2 = temp_2.dropna(subset=['subject_id-2']) # Drop cases where no statin is prescribed as we are only interested to know whether statin prescribed during hospitalization. (1028, 26)
temp_2.sort_values(by=['subject_id','intime'],ascending=True,inplace=True) #(1028, 26)
# Add ICUintime and ICUadmittime (for current ICU stay) from temp_subset for comparison later
temp_2 = pd.merge(temp_2,temp_subset[['subject_id','ICUintime','ICUadmittime']],how='left',on='subject_id') # (1028, 28)
temp_2.to_csv('E:/MSBA/Semester 2/SPH5104 Analytics for Better Health/Group Assignments/Dataset/Codes/Data Preparation/New data_From 4April/Temp/temp_2.csv',index=False)
temp_2.shape
# (1028, 28)

(1028, 28)

In [52]:
temp_2

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,subject_id-2,hadm_id-2,...,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,admittime,ICUintime,ICUadmittime
0,10002155,20345487,32358465,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-03-09 21:33:00,2131/3/10 18:09,0.858576,10002155.0,20345487.0,...,NaN,80.0,mg,2.0,TAB,1.0,PO/NG,2131-03-09 20:33:00,2131-03-09 21:33:00,2131-03-09 20:33:00
1,10029057,28919637,36004746,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2187-07-30 21:48:00,2187/8/3 3:17,3.228947,10029057.0,28919637.0,...,NaN,20.0,mg,1.0,TAB,1.0,PO/NG,2187-07-30 20:43:00,2187-07-30 21:48:00,2187-07-30 20:43:00
2,10034317,20827960,36228864,Trauma SICU (TSICU),Trauma SICU (TSICU),2159-08-26 12:59:00,2159/8/29 19:54,3.287708,10034317.0,20827960.0,...,NaN,20.0,mg,0.5,TAB,1.0,PO,2159-08-26 00:00:00,2159-08-30 12:35:00,2159-08-26 00:00:00
3,10034317,20827960,36228864,Trauma SICU (TSICU),Trauma SICU (TSICU),2159-08-26 12:59:00,2159/8/29 19:54,3.287708,10034317.0,20827960.0,...,NaN,20.0,mg,1.0,TAB,1.0,PO,2159-08-26 00:00:00,2159-08-30 12:35:00,2159-08-26 00:00:00
4,10034317,20827960,33536886,Trauma SICU (TSICU),Trauma SICU (TSICU),2159-08-30 12:35:00,2159/8/31 16:26,1.160567,10034317.0,20827960.0,...,NaN,20.0,mg,0.5,TAB,1.0,PO,2159-08-26 00:00:00,2159-08-30 12:35:00,2159-08-26 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,19971771,23652398,37070096,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2117-04-17 16:29:00,2117/4/19 17:55,2.059421,19971771.0,23652398.0,...,NaN,20.0,mg,2.0,TAB,1.0,PO/NG,2117-04-10 20:28:00,2117-04-26 01:49:00,2117-04-26 00:21:00
1024,19971771,26230047,38668070,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2117-04-26 01:49:00,2117/5/21 13:25,25.483461,19971771.0,26230047.0,...,NaN,20.0,mg,2.0,TAB,1.0,PO/NG,2117-04-26 00:21:00,2117-04-26 01:49:00,2117-04-26 00:21:00
1025,19987702,26568899,34859288,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2131-01-15 22:29:00,2131/1/17 21:26,1.956030,19987702.0,26568899.0,...,NaN,80.0,mg,1.0,TAB,1.0,PO/NG,2131-01-13 16:45:00,2131-01-15 22:29:00,2131-01-13 16:45:00
1026,19996783,21880161,31700331,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2188-05-10 02:20:00,2188/5/10 22:30,0.840116,19996783.0,21880161.0,...,NaN,80.0,mg,1.0,TAB,1.0,PO/NG,2188-05-09 15:55:00,2188-05-10 02:20:00,2188-05-09 15:55:00


In [53]:
'''
Filter patients who have statin prescribed in PREVIOUS hospitalizations.

On the temp dataset, we will use groupby by subject_id
For all the rows related to a particular subject_id, we will only keep rows where the admittime is earlier than the ICUadmittime in our Troponin (obtained on 8April)
This means that  this statin Rx happens in previous hospitalization (vs. current ICU stay) -> patients acutally have prior exposure to statin
'''

temp_group_2 = temp_2.groupby('subject_id')

cols_2 = temp_2.columns

# temp_df_2 is created to store the data 
temp_df_2 = pd.DataFrame(columns=cols_2)


# In the loop -> we will only keep a row if hospital admission time is earlier than the current hospital admission time
for name, group in temp_group_2:
    group = group.loc[group['admittime']<group['ICUadmittime']] # Must be previous hospitalization 
    temp_df_2 = pd.concat([temp_df_2,group],axis=0)
temp_df_2.shape # (38, 28)


(38, 28)

In [54]:
count = pd.DataFrame(temp_df_2['subject_id'].value_counts())
count.shape # 26 Distinct patients

(26, 1)

In [55]:
# Check the original grouping in the dataset
df_3['statin'].value_counts()

0    979
2    531
1    121
Name: statin, dtype: int64

In [56]:
# For patients in temp_df_2, they have received statins in previous hospitalization. 
# Thus we classify them as Group 1 as well (statin exposed)

df_3['statin'] = np.where(df_3['subject_id'].isin(temp_df_2['subject_id']),1,df_3['statin'])

In [57]:
# Check the updated grouping in oru dataset
df_3['statin'].value_counts()

0    979
2    505
1    147
Name: statin, dtype: int64

In [ ]:
# # Obtain the latest dataset
df_3.to_csv('Final data_9April.csv',index=False)